<a href="https://colab.research.google.com/github/jetsonmom/6.23_automobility_lesson/blob/main/6_23_%EA%B0%95%EC%9D%98_step3_%EC%9E%A5%EC%95%A0%EB%AC%BC_%ED%9A%8C%ED%94%BC_%26_%EC%8B%A4%EC%A0%9C%EC%98%81%EC%83%81%EB%8F%84%EB%A1%9C_%EC%8B%9C%EB%AE%AC%EB%A0%88%EC%9D%B4%EC%85%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

장애물 회피 시뮬레이션

In [ ]:
# 첫 번째 셀 - 라이브러리 설치
!pip install opencv-python
!pip install requests
!pip install pillow
!pip install matplotlib
!pip install numpy
!pip install IPython

# 애니메이션이 안 되면 추가 설치
!apt-get update
!apt-get install -y ffmpeg

In [ ]:
# 🚧 장애물 회피 자율주행 시뮬레이션
# 실제 자율주행차 수준의 장애물 인식 및 회피

# 두 번째 셀 - Import
import cv2
import requests
from PIL import Image  # ← 여기! "Imag"가 아니라 "Image"
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import random
import math
import urllib.request
import io

print("✅ 모든 라이브러리 준비 완료!")

# 설정
plt.style.use('dark_background')
plt.rcParams['figure.facecolor'] = 'black'

class ObstacleAvoidanceSimulator:
    def __init__(self):
        """장애물 회피 시뮬레이터"""
        # 기본 설정
        self.road_width = 100
        self.car_x = 50
        self.car_y = 10
        self.car_speed = 1.5
        self.car_heading = 0

        # 차선 정보
        self.left_lane = 35
        self.center_lane = 50
        self.right_lane = 65
        self.current_target_lane = self.center_lane

        # 제어 변수
        self.steering_angle = 0
        self.max_steering = 25

        # PID 파라미터
        self.kp = 2.0
        self.ki = 0.05
        self.kd = 1.0

        # PID 상태
        self.previous_error = 0
        self.integral_error = 0

        # 🚧 장애물 관리
        self.obstacles = []
        self.obstacle_spawn_timer = 0
        self.detection_range = 30  # 장애물 감지 거리

        # 🧠 행동 상태
        self.behavior_state = "LANE_FOLLOWING"  # LANE_FOLLOWING, AVOIDING, RETURNING
        self.avoidance_timer = 0
        self.original_lane = self.center_lane

        # 기록용
        self.positions = []
        self.obstacles_history = []
        self.behavior_history = []
        self.safety_distances = []

        print("🚧 장애물 회피 시뮬레이터 준비!")
        print("🤖 AI 기반 장애물 인식 및 회피 시스템 활성화!")

    class Obstacle:
        """장애물 클래스"""
        def __init__(self, x, y, obstacle_type="car"):
            self.x = x
            self.y = y
            self.type = obstacle_type
            self.speed = random.uniform(0.5, 1.0)  # 장애물 속도
            self.width = 3
            self.height = 6
            self.active = True

            # 장애물 타입별 특성
            if obstacle_type == "car":
                self.color = 'red'
                self.marker = 's'  # 사각형
                self.size = 150
            elif obstacle_type == "truck":
                self.color = 'orange'
                self.marker = 's'
                self.size = 200
                self.width = 4
                self.height = 8
            elif obstacle_type == "debris":
                self.color = 'gray'
                self.marker = 'X'
                self.size = 100
                self.speed = 0  # 정지된 장애물

        def update(self):
            """장애물 위치 업데이트"""
            if self.active:
                self.y += self.speed

                # 화면을 벗어나면 비활성화
                if self.y > 200:
                    self.active = False

        def get_distance_to(self, car_x, car_y):
            """차량과의 거리 계산"""
            return math.sqrt((self.x - car_x)**2 + (self.y - car_y)**2)

    def spawn_obstacles(self, frame):
        """장애물 생성"""
        self.obstacle_spawn_timer += 1

        # 랜덤하게 장애물 생성 (난이도 조절)
        if self.obstacle_spawn_timer > 80:  # 8초마다
            self.obstacle_spawn_timer = 0

            # 장애물 타입 랜덤 선택
            obstacle_types = ["car", "car", "truck", "debris"]
            obstacle_type = random.choice(obstacle_types)

            # 차선 랜덤 선택 (현재 차량 위치 제외)
            lanes = [self.left_lane, self.center_lane, self.right_lane]
            # 현재 차량 차선 제외하고 선택
            available_lanes = [lane for lane in lanes if abs(lane - self.car_x) > 10]

            if available_lanes:
                obstacle_lane = random.choice(available_lanes)
                obstacle_y = self.car_y + random.uniform(40, 80)  # 앞쪽에 생성

                new_obstacle = self.Obstacle(obstacle_lane, obstacle_y, obstacle_type)
                self.obstacles.append(new_obstacle)

                print(f"🚧 {obstacle_type.upper()} 장애물 생성: 차선 {obstacle_lane}, 거리 {obstacle_y - self.car_y:.1f}m")

    def detect_obstacles(self):
        """장애물 감지 시스템 (센서 시뮬레이션)"""
        detected_obstacles = []

        for obstacle in self.obstacles:
            if not obstacle.active:
                continue

            # 전방 감지 범위 내 장애물 찾기
            distance = obstacle.get_distance_to(self.car_x, self.car_y)

            # 전방 감지 조건
            is_ahead = obstacle.y > self.car_y
            is_in_range = distance < self.detection_range
            is_in_path = abs(obstacle.x - self.car_x) < 20  # 진행 경로상

            if is_ahead and is_in_range and is_in_path:
                detected_obstacles.append({
                    'obstacle': obstacle,
                    'distance': distance,
                    'lateral_distance': abs(obstacle.x - self.car_x),
                    'time_to_collision': distance / max(self.car_speed, 0.1)
                })

        # 거리순 정렬 (가장 가까운 것부터)
        detected_obstacles.sort(key=lambda x: x['distance'])

        return detected_obstacles

    def behavior_planner(self, detected_obstacles):
        """행동 계획 AI (실제 자율주행차 로직)"""
        # 🚨 위험 상황 평가
        critical_distance = 15  # 위험 거리
        safe_distance = 25      # 안전 거리

        if not detected_obstacles:
            # 장애물 없음 - 원래 차선으로 복귀
            if self.behavior_state == "AVOIDING":
                self.behavior_state = "RETURNING"
                print("✅ 장애물 없음 - 원래 차선으로 복귀 시작")
            elif self.behavior_state == "RETURNING":
                if abs(self.car_x - self.original_lane) < 1.0:
                    self.behavior_state = "LANE_FOLLOWING"
                    print("🎯 원래 차선 복귀 완료")

            self.current_target_lane = self.original_lane
            return

        # 가장 위험한 장애물 분석
        closest_obstacle = detected_obstacles[0]
        distance = closest_obstacle['distance']
        obstacle = closest_obstacle['obstacle']

        # 🚨 긴급 회피 필요
        if distance < critical_distance:
            self.behavior_state = "AVOIDING"

            # 회피 방향 결정 (좌우 중 안전한 쪽)
            if obstacle.x < self.car_x:
                # 장애물이 왼쪽 → 오른쪽으로 회피
                if self.current_target_lane < self.right_lane:
                    self.current_target_lane = self.right_lane
                    print("🚨 긴급 우회 - 오른쪽 차선으로!")
            else:
                # 장애물이 오른쪽 → 왼쪽으로 회피
                if self.current_target_lane > self.left_lane:
                    self.current_target_lane = self.left_lane
                    print("🚨 긴급 우회 - 왼쪽 차선으로!")

        # ⚠️ 예방적 회피
        elif distance < safe_distance and self.behavior_state == "LANE_FOLLOWING":
            self.behavior_state = "AVOIDING"

            # 더 안전한 회피 방향 선택
            left_safe = self.check_lane_safety(self.left_lane)
            right_safe = self.check_lane_safety(self.right_lane)

            if obstacle.x <= self.center_lane and right_safe:
                self.current_target_lane = self.right_lane
                print("⚠️ 예방적 우회 - 오른쪽 차선으로")
            elif obstacle.x >= self.center_lane and left_safe:
                self.current_target_lane = self.left_lane
                print("⚠️ 예방적 우회 - 왼쪽 차선으로")

    def check_lane_safety(self, target_lane):
        """목표 차선의 안전성 검사"""
        for obstacle in self.obstacles:
            if not obstacle.active:
                continue

            # 목표 차선에 장애물이 있는지 확인
            if (abs(obstacle.x - target_lane) < 8 and
                abs(obstacle.y - self.car_y) < 20):
                return False
        return True

    def advanced_pid_controller(self):
        """장애물 회피용 강화된 PID 제어"""
        # 현재 오차
        current_error = self.current_target_lane - self.car_x

        # 🚨 긴급 상황 시 게인 증가
        if self.behavior_state == "AVOIDING":
            kp_adjusted = self.kp * 1.5  # 더 빠른 반응
            kd_adjusted = self.kd * 1.2  # 안정성 증가
        else:
            kp_adjusted = self.kp
            kd_adjusted = self.kd

        # PID 계산
        proportional = kp_adjusted * current_error

        self.integral_error += current_error * 0.1
        self.integral_error = np.clip(self.integral_error, -20, 20)
        integral = self.ki * self.integral_error

        error_derivative = (current_error - self.previous_error) / 0.1
        derivative = kd_adjusted * error_derivative

        # 최종 조향각
        self.steering_angle = proportional + integral + derivative

        # 긴급 상황 시 조향 범위 확대
        max_steering = self.max_steering * 1.3 if self.behavior_state == "AVOIDING" else self.max_steering
        self.steering_angle = np.clip(self.steering_angle, -max_steering, max_steering)

        self.previous_error = current_error

        return self.steering_angle, current_error

    def adaptive_speed_control(self, detected_obstacles):
        """적응적 속도 제어"""
        if not detected_obstacles:
            target_speed = 1.5  # 정상 속도
        else:
            closest_distance = detected_obstacles[0]['distance']

            if closest_distance < 10:
                target_speed = 0.8  # 크게 감속
            elif closest_distance < 20:
                target_speed = 1.0  # 감속
            elif closest_distance < 30:
                target_speed = 1.2  # 약간 감속
            else:
                target_speed = 1.5  # 정상 속도

        # 부드러운 속도 변화
        speed_diff = target_speed - self.car_speed
        self.car_speed += speed_diff * 0.2

        return self.car_speed

    def update_vehicle_physics(self):
        """차량 물리 업데이트"""
        # 조향각에 따른 이동
        steering_rad = np.radians(self.steering_angle)
        wheelbase = 2.5
        dt = 0.1

        self.car_heading += (self.car_speed * np.tan(steering_rad) / wheelbase) * dt
        self.car_heading = np.clip(self.car_heading, -np.pi/3, np.pi/3)

        # 위치 업데이트
        self.car_x += self.car_speed * np.sin(self.car_heading) * dt
        self.car_y += self.car_speed * np.cos(self.car_heading) * dt

        # 경계 체크
        self.car_x = np.clip(self.car_x, 25, 75)

        # 화면 스크롤
        if self.car_y > 150:
            self.car_y = 10
            # 장애물들도 상대적으로 이동
            for obstacle in self.obstacles:
                obstacle.y -= 140
                if obstacle.y < 0:
                    obstacle.active = False

    def create_obstacle_avoidance_visualization(self):
        """장애물 회피 시각화"""
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 12))
        fig.patch.set_facecolor('black')

        # === 메인 시뮬레이션 (좌상) ===
        ax1.set_xlim(15, 85)
        ax1.set_ylim(0, 150)
        ax1.set_facecolor('black')
        ax1.set_title('🚧 Obstacle Avoidance Simulation',
                     fontsize=16, color='white', fontweight='bold')

        # 도로
        road_surface = plt.Rectangle((20, 0), 60, 150,
                                   facecolor='#2a2a2a', alpha=0.9)
        ax1.add_patch(road_surface)

        # 도로 경계
        ax1.plot([20, 20], [0, 150], 'gold', linewidth=4)
        ax1.plot([80, 80], [0, 150], 'gold', linewidth=4)

        # 차선
        for y in range(0, 150, 10):
            ax1.plot([self.left_lane, self.left_lane], [y, y+5], 'white', linewidth=3, alpha=0.8)
            ax1.plot([self.center_lane, self.center_lane], [y, y+5], 'white', linewidth=3, alpha=0.8)
            ax1.plot([self.right_lane, self.right_lane], [y, y+5], 'white', linewidth=3, alpha=0.8)

        # 차량
        self.car_dot, = ax1.plot(self.car_x, self.car_y, 'bo', markersize=18,
                                markeredgecolor='cyan', markeredgewidth=3,
                                label='AI Vehicle', zorder=10)

        # 차량 경로
        self.path_line, = ax1.plot([], [], 'cyan', linewidth=3, alpha=0.8,
                                  label='Vehicle Path', zorder=5)

        # 목표 차선
        self.target_line, = ax1.plot([], [], 'lime', linewidth=3, alpha=0.7,
                                    linestyle=':', label='Target Lane', zorder=6)

        # 감지 범위
        self.detection_circle = plt.Circle((self.car_x, self.car_y), self.detection_range,
                                         fill=False, color='yellow', alpha=0.3, linestyle='--',
                                         label='Detection Range')
        ax1.add_patch(self.detection_circle)

        # 장애물들 (빈 플롯으로 시작)
        self.obstacle_plots = []

        ax1.legend(loc='upper right')
        ax1.grid(True, alpha=0.2)
        ax1.set_xlabel('Lane Position (m)', color='white')
        ax1.set_ylabel('Distance (m)', color='white')

        # === AI 대시보드 (우상) ===
        ax2.set_facecolor('black')
        ax2.set_title('🤖 AI Control Dashboard', fontsize=16, color='white', fontweight='bold')
        ax2.set_xlim(0, 10)
        ax2.set_ylim(0, 10)

        self.ai_dashboard_text = ax2.text(0.5, 5, '', fontsize=10, color='cyan',
                                         family='monospace', ha='left', va='center')
        ax2.axis('off')

        # === 장애물 분석 (좌하) ===
        ax3.set_facecolor('black')
        ax3.set_title('🔍 Obstacle Detection', fontsize=14, color='white', fontweight='bold')
        ax3.set_xlabel('Time Steps', color='white')
        ax3.set_ylabel('Distance (m)', color='white')
        ax3.grid(True, alpha=0.3)

        # 안전 거리 기준선
        ax3.axhline(y=25, color='green', linestyle='--', alpha=0.5, label='Safe Distance')
        ax3.axhline(y=15, color='red', linestyle='--', alpha=0.5, label='Critical Distance')

        self.obstacle_distance_plot, = ax3.plot([], [], 'red', linewidth=2, label='Closest Obstacle')
        ax3.legend()
        ax3.set_xlim(0, 100)
        ax3.set_ylim(0, 50)

        # === 행동 분석 (우하) ===
        ax4.set_facecolor('black')
        ax4.set_title('🧠 Behavior Analysis', fontsize=14, color='white', fontweight='bold')
        ax4.set_xlabel('Time Steps', color='white')
        ax4.set_ylabel('Control Values', color='white')
        ax4.grid(True, alpha=0.3)

        self.steering_plot, = ax4.plot([], [], 'yellow', linewidth=2, label='Steering')
        self.speed_plot, = ax4.plot([], [], 'green', linewidth=2, label='Speed')
        ax4.legend()
        ax4.set_xlim(0, 100)
        ax4.set_ylim(-30, 30)

        return fig, ax1, ax2, ax3, ax4

    def animate(self, frame):
        """애니메이션 업데이트"""
        # 1. 장애물 생성
        self.spawn_obstacles(frame)

        # 2. 장애물 위치 업데이트
        for obstacle in self.obstacles:
            obstacle.update()

        # 3. 장애물 감지
        detected_obstacles = self.detect_obstacles()

        # 4. 행동 계획
        self.behavior_planner(detected_obstacles)

        # 5. 제어 실행
        steering, error = self.advanced_pid_controller()
        speed = self.adaptive_speed_control(detected_obstacles)

        # 6. 차량 업데이트
        self.update_vehicle_physics()

        # 7. 데이터 기록
        self.positions.append([self.car_x, self.car_y])
        self.behavior_history.append(self.behavior_state)

        closest_distance = detected_obstacles[0]['distance'] if detected_obstacles else 50
        self.safety_distances.append(closest_distance)

        # 기록 길이 제한
        max_history = 100
        for history in [self.positions, self.behavior_history, self.safety_distances]:
            if len(history) > max_history:
                history.pop(0)

        # 8. 시각화 업데이트

        # 차량 위치
        self.car_dot.set_data([self.car_x], [self.car_y])

        # 차량 경로
        if len(self.positions) > 5:
            recent_positions = self.positions[-30:]
            path_x = [pos[0] for pos in recent_positions]
            path_y = [pos[1] for pos in recent_positions]
            self.path_line.set_data(path_x, path_y)

        # 목표 차선
        target_y = [self.car_y-10, self.car_y+40]
        target_x = [self.current_target_lane, self.current_target_lane]
        self.target_line.set_data(target_x, target_y)

        # 감지 범위 업데이트
        self.detection_circle.center = (self.car_x, self.car_y)

        # 장애물 표시
        # 기존 장애물 플롯 제거
        for plot in self.obstacle_plots:
            if plot in self.ax1.collections:
                plot.remove()
        self.obstacle_plots.clear()

        # 활성 장애물들 다시 그리기
        for obstacle in self.obstacles:
            if obstacle.active:
                scatter = self.ax1.scatter(obstacle.x, obstacle.y,
                                        c=obstacle.color, marker=obstacle.marker,
                                        s=obstacle.size, alpha=0.8, zorder=8,
                                        edgecolors='white', linewidth=2)
                self.obstacle_plots.append(scatter)

        # AI 대시보드 업데이트
        behavior_emoji = {
            "LANE_FOLLOWING": "🛣️",
            "AVOIDING": "🚨",
            "RETURNING": "↩️"
        }

        target_lane_name = {
            self.left_lane: "Left",
            self.center_lane: "Center",
            self.right_lane: "Right"
        }.get(self.current_target_lane, "Unknown")

        ai_info = f"""🤖 AI AUTONOMOUS SYSTEM

{behavior_emoji.get(self.behavior_state, "🤖")} {self.behavior_state}

🎯 Target: {target_lane_name} Lane
📍 Position: ({self.car_x:.1f}, {self.car_y:.1f})
🏃 Speed: {speed:.1f} m/s
🎛️ Steering: {steering:.1f}°

🚧 OBSTACLE STATUS
Detected: {len(detected_obstacles)}
Closest: {closest_distance:.1f}m
Safety: {"🟢 SAFE" if closest_distance > 25 else "🟡 CAUTION" if closest_distance > 15 else "🔴 DANGER"}

⚙️ CONTROL MODE
Error: {error:.2f}m
Behavior: {self.behavior_state}

⏱️ Frame: {frame}
        """
        self.ai_dashboard_text.set_text(ai_info)

        # 그래프 업데이트
        if len(self.safety_distances) > 1:
            time_steps = list(range(len(self.safety_distances)))
            self.obstacle_distance_plot.set_data(time_steps, self.safety_distances)

            # 조향각과 속도 (스케일 조정)
            steering_history = [steering] * len(time_steps)  # 현재 값으로 임시
            speed_history = [speed * 10] * len(time_steps)   # 스케일 조정
            self.steering_plot.set_data(time_steps, steering_history)
            self.speed_plot.set_data(time_steps, speed_history)

        return_objects = [self.car_dot, self.path_line, self.target_line,
                         self.ai_dashboard_text, self.obstacle_distance_plot,
                         self.steering_plot, self.speed_plot] + self.obstacle_plots

        return tuple(return_objects)

# 전역 변수로 ax1 저장
global_ax1 = None

def set_global_ax1(ax1):
    global global_ax1
    global_ax1 = ax1

# === 실행 ===
print("🚧 Obstacle Avoidance Simulation 시작!")
print("🤖 AI 기반 실시간 장애물 감지 및 회피!")
print("🚨 긴급 상황 대응 시스템 활성화!")

simulator = ObstacleAvoidanceSimulator()
fig, ax1, ax2, ax3, ax4 = simulator.create_obstacle_avoidance_visualization()

# ax1을 시뮬레이터에 저장
simulator.ax1 = ax1

# 애니메이션 생성
anim = animation.FuncAnimation(
    fig, simulator.animate,
    interval=100,      # 100ms 간격
    frames=2500,       # 긴 시뮬레이션
    blit=False,
    repeat=True
)

plt.tight_layout()
plt.show()

# MP4 저장
def save_obstacle_avoidance_mp4():
    print("\n💾 장애물 회피 시뮬레이션 저장 중...")
    try:
        Writer = animation.writers['ffmpeg']
        writer = Writer(fps=10, bitrate=2500)
        anim.save('obstacle_avoidance_simulation.mp4', writer=writer)

        from google.colab import files
        files.download('obstacle_avoidance_simulation.mp4')
        print("✅ 장애물 회피 비디오 저장 완료!")
    except Exception as e:
        print(f"❌ 저장 실패: {e}")

save_obstacle_avoidance_mp4()

print("\n🚧 장애물 회피 시스템 특징:")
print("✅ 실시간 장애물 감지 (30m 범위)")
print("✅ AI 행동 계획 (차선 변경 결정)")
print("✅ 긴급/예방적 회피 모드")
print("✅ 다양한 장애물 타입 (차량, 트럭, 잔해)")
print("✅ 적응적 속도 제어")
print("✅ 안전 거리 유지")
print("\n🚀 진짜 자율주행차 수준의 장애물 회피!")

실제도로영상

In [ ]:
# 유튜브나 인터넷에서 도로 영상 다운로드
video_url = "실제_고속도로_영상.mp4"
# 또는 미리 준비된 도로 영상 사용

In [ ]:
# 우리가 이미 배운 기술들!
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, 50, 150)
lines = cv2.HoughLinesP(edges, ...)
# → 실제 영상에서 차선 찾기!

In [ ]:
# 검출된 차선 정보로 PID 제어
target_lane = detected_center_line
error = target_lane - virtual_car_position
steering = pid_controller(error)
# → 가상 차량이 실제 차선 따라 움직임!

In [ ]:
# 📹 실제 도로 영상 기반 자율주행 시뮬레이션
# 진짜 도로 영상에서 차선 검출 + 가상 차량 주행

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import cv2
from IPython.display import HTML
import requests
from PIL import Image
import io

# 설정
plt.style.use('dark_background')
plt.rcParams['figure.facecolor'] = 'black'

class RealVideoLaneSimulator:
    def __init__(self):
        """실제 영상 기반 시뮬레이터"""
        # 영상 설정
        self.current_frame = None
        self.frame_width = 640
        self.frame_height = 480

        # 가상 차량 설정
        self.car_x = 320  # 영상 중앙
        self.car_y = 400  # 화면 하단
        self.car_speed = 1.0

        # 차선 검출 결과
        self.detected_lanes = []
        self.target_lane_x = 320  # 목표 차선 X 좌표

        # PID 제어
        self.steering_angle = 0
        self.kp = 0.5
        self.ki = 0.01
        self.kd = 0.3
        self.previous_error = 0
        self.integral_error = 0

        # 기록용
        self.positions = []
        self.steering_history = []
        self.lane_errors = []

        # 샘플 도로 이미지들 (인터넷에서 가져올 예정)
        self.road_images = []
        self.current_image_index = 0

        print("📹 실제 도로 영상 시뮬레이터 준비!")
        print("🚗 가상 차량이 실제 도로에서 주행합니다!")

    def download_sample_road_images(self):
        """샘플 도로 이미지 다운로드"""
        print("📥 도로 이미지 다운로드 중...")

        # 무료 도로 이미지 URL들 (Unsplash)
        image_urls = [
            "https://images.unsplash.com/photo-1449824913935-59a10b8d2000?w=640&h=480&fit=crop",
            "https://images.unsplash.com/photo-1558618666-fcd25c85cd64?w=640&h=480&fit=crop",
            "https://images.unsplash.com/photo-1465056836041-7f43ac27dcb5?w=640&h=480&fit=crop",
            "https://images.unsplash.com/photo-1449824913935-59a10b8d2000?w=640&h=480&fit=crop&rotate=5",
            "https://images.unsplash.com/photo-1558618666-fcd25c85cd64?w=640&h=480&fit=crop&rotate=-3"
        ]

        for i, url in enumerate(image_urls):
            try:
                response = requests.get(url, timeout=10)
                img = Image.open(io.BytesIO(response.content))
                img = img.resize((self.frame_width, self.frame_height))

                # PIL to OpenCV
                img_array = np.array(img)
                if len(img_array.shape) == 3:
                    img_array = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)

                self.road_images.append(img_array)
                print(f"✅ 도로 이미지 {i+1} 다운로드 완료")

            except Exception as e:
                print(f"⚠️ 이미지 {i+1} 다운로드 실패: {e}")
                # 실패 시 더미 이미지 생성
                dummy_img = self.create_dummy_road_image()
                self.road_images.append(dummy_img)

        if not self.road_images:
            # 모든 다운로드 실패 시 더미 이미지들 생성
            print("📸 더미 도로 이미지 생성 중...")
            for i in range(5):
                dummy_img = self.create_dummy_road_image(variation=i)
                self.road_images.append(dummy_img)

        print(f"🎯 총 {len(self.road_images)}개 도로 이미지 준비 완료!")

    def create_dummy_road_image(self, variation=0):
        """더미 도로 이미지 생성"""
        img = np.zeros((self.frame_height, self.frame_width, 3), dtype=np.uint8)

        # 도로 배경 (어두운 회색)
        img[:, :] = [40, 40, 40]

        # 도로 영역 (더 어두운 회색)
        road_start = self.frame_width // 4
        road_end = 3 * self.frame_width // 4
        img[:, road_start:road_end] = [20, 20, 20]

        # 차선 그리기 (variation에 따라 다르게)
        center_x = self.frame_width // 2

        if variation % 2 == 0:  # 직선
            offset = (variation - 2) * 20
            left_lane = center_x - 60 + offset
            right_lane = center_x + 60 + offset
        else:  # 곡선
            for y in range(0, self.frame_height, 10):
                curve = int(30 * np.sin(y * 0.01 + variation))
                left_lane = center_x - 60 + curve
                right_lane = center_x + 60 + curve

                # 차선 그리기
                if 0 <= left_lane < self.frame_width:
                    cv2.line(img, (left_lane, y), (left_lane, y+5), (255, 255, 255), 3)
                if 0 <= right_lane < self.frame_width:
                    cv2.line(img, (right_lane, y), (right_lane, y+5), (255, 255, 255), 3)
            return img

        # 직선 차선
        for y in range(0, self.frame_height, 15):
            if 0 <= left_lane < self.frame_width:
                cv2.line(img, (left_lane, y), (left_lane, y+8), (255, 255, 255), 4)
            if 0 <= right_lane < self.frame_width:
                cv2.line(img, (right_lane, y), (right_lane, y+8), (255, 255, 255), 4)

        return img

    def detect_lanes_in_frame(self, frame):
        """실제 차선 검출 (우리가 배운 알고리즘!)"""
        # 1. 그레이스케일 변환
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # 2. 가우시안 블러
        blur = cv2.GaussianBlur(gray, (5, 5), 0)

        # 3. 엣지 검출
        edges = cv2.Canny(blur, 50, 150)

        # 4. 관심 영역 설정 (ROI) - 하단 절반
        height, width = edges.shape
        mask = np.zeros_like(edges)

        # 사다리꼴 모양 ROI
        vertices = np.array([
            [(width*0.1, height),           # 왼쪽 아래
             (width*0.4, height*0.6),       # 왼쪽 위
             (width*0.6, height*0.6),       # 오른쪽 위
             (width*0.9, height)]           # 오른쪽 아래
        ], dtype=np.int32)

        cv2.fillPoly(mask, vertices, 255)
        roi_edges = cv2.bitwise_and(edges, mask)

        # 5. 허프 변환으로 직선 검출
        lines = cv2.HoughLinesP(
            roi_edges,
            rho=1,
            theta=np.pi/180,
            threshold=30,
            minLineLength=50,
            maxLineGap=100
        )

        # 6. 차선 분류 및 평균화
        left_lines = []
        right_lines = []

        if lines is not None:
            for line in lines:
                x1, y1, x2, y2 = line[0]

                # 기울기 계산
                if x2 - x1 == 0:
                    continue
                slope = (y2 - y1) / (x2 - x1)

                # 기울기로 좌우 구분
                if slope < -0.3:  # 왼쪽 차선
                    left_lines.append(line[0])
                elif slope > 0.3:  # 오른쪽 차선
                    right_lines.append(line[0])

        # 7. 차선 평균화 및 목표 차선 계산
        lane_info = {
            'left_lane': None,
            'right_lane': None,
            'center_lane': None,
            'roi_edges': roi_edges
        }

        if left_lines:
            left_lane = self.average_lines(left_lines, height)
            lane_info['left_lane'] = left_lane

        if right_lines:
            right_lane = self.average_lines(right_lines, height)
            lane_info['right_lane'] = right_lane

        # 중앙 차선 계산
        if lane_info['left_lane'] and lane_info['right_lane']:
            left_x = lane_info['left_lane'][0]
            right_x = lane_info['right_lane'][0]
            center_x = (left_x + right_x) // 2
            lane_info['center_lane'] = (center_x, height-50, center_x, height*0.6)
            self.target_lane_x = center_x
        elif lane_info['left_lane']:
            # 왼쪽 차선만 있으면 오른쪽으로 추정
            left_x = lane_info['left_lane'][0]
            center_x = left_x + 120  # 추정
            self.target_lane_x = center_x
        elif lane_info['right_lane']:
            # 오른쪽 차선만 있으면 왼쪽으로 추정
            right_x = lane_info['right_lane'][0]
            center_x = right_x - 120  # 추정
            self.target_lane_x = center_x

        self.detected_lanes = lane_info
        return lane_info

    def average_lines(self, lines, height):
        """차선들을 평균화하여 하나의 선으로 만들기"""
        x_coords = []
        y_coords = []

        for line in lines:
            x1, y1, x2, y2 = line
            x_coords.extend([x1, x2])
            y_coords.extend([y1, y2])

        if len(x_coords) < 2:
            return None

        # 최소자승법으로 직선 피팅
        poly = np.polyfit(y_coords, x_coords, 1)

        # Y 좌표 범위에서 X 좌표 계산
        y1 = height
        y2 = int(height * 0.6)
        x1 = int(poly[0] * y1 + poly[1])
        x2 = int(poly[0] * y2 + poly[1])

        return (x1, y1, x2, y2)

    def pid_controller(self):
        """PID 제어로 조향각 계산"""
        # 현재 오차 (목표 차선 - 현재 차량 위치)
        error = self.target_lane_x - self.car_x

        # PID 계산
        proportional = self.kp * error

        self.integral_error += error * 0.1
        self.integral_error = np.clip(self.integral_error, -100, 100)
        integral = self.ki * self.integral_error

        derivative = self.kd * (error - self.previous_error) / 0.1

        # 최종 조향각
        self.steering_angle = proportional + integral + derivative
        self.steering_angle = np.clip(self.steering_angle, -50, 50)

        self.previous_error = error

        return error

    def update_virtual_vehicle(self):
        """가상 차량 위치 업데이트"""
        # 조향각에 따른 횡방향 이동
        lateral_movement = self.steering_angle * 0.05

        # 차량 위치 업데이트
        self.car_x += lateral_movement

        # 화면 경계 체크
        self.car_x = np.clip(self.car_x, 50, self.frame_width - 50)

        # 영상 프레임 변경 (전진 효과)
        if len(self.road_images) > 1:
            self.current_image_index = (self.current_image_index + 1) % len(self.road_images)

    def draw_lane_detection_overlay(self, frame):
        """차선 검출 결과를 영상에 오버레이"""
        overlay = frame.copy()

        # 검출된 차선 그리기
        if self.detected_lanes.get('left_lane'):
            x1, y1, x2, y2 = self.detected_lanes['left_lane']
            cv2.line(overlay, (x1, y1), (x2, y2), (0, 255, 0), 5)  # 초록색

        if self.detected_lanes.get('right_lane'):
            x1, y1, x2, y2 = self.detected_lanes['right_lane']
            cv2.line(overlay, (x1, y1), (x2, y2), (0, 255, 0), 5)  # 초록색

        if self.detected_lanes.get('center_lane'):
            x1, y1, x2, y2 = self.detected_lanes['center_lane']
            cv2.line(overlay, (x1, y1), (x2, y2), (0, 255, 255), 3)  # 노란색 점선

        # 가상 차량 그리기
        car_size = 15
        cv2.circle(overlay, (int(self.car_x), int(self.car_y)), car_size, (0, 0, 255), -1)  # 빨간 원
        cv2.circle(overlay, (int(self.car_x), int(self.car_y)), car_size, (255, 255, 255), 3)  # 하얀 테두리

        # 차량 방향 표시 (조향각)
        direction_length = 30
        direction_x = int(self.car_x + direction_length * np.sin(np.radians(self.steering_angle)))
        direction_y = int(self.car_y - direction_length * np.cos(np.radians(self.steering_angle)))
        cv2.arrowedLine(overlay, (int(self.car_x), int(self.car_y)),
                       (direction_x, direction_y), (255, 0, 255), 3)

        # 목표 지점 표시
        cv2.circle(overlay, (int(self.target_lane_x), int(self.car_y - 50)), 8, (0, 255, 255), -1)

        return overlay

    def create_real_video_visualization(self):
        """실제 영상 기반 시각화"""
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 12))
        fig.patch.set_facecolor('black')

        # === 실제 도로 영상 + 차선 검출 (좌상) ===
        ax1.set_title('📹 Real Road Video + Lane Detection',
                     fontsize=16, color='white', fontweight='bold')
        ax1.axis('off')

        # 초기 이미지 (더미)
        dummy_frame = np.zeros((self.frame_height, self.frame_width, 3), dtype=np.uint8)
        self.video_display = ax1.imshow(dummy_frame, aspect='auto')

        # === 엣지 검출 결과 (우상) ===
        ax2.set_title('🔍 Edge Detection (Canny)',
                     fontsize=16, color='white', fontweight='bold')
        ax2.axis('off')

        dummy_edges = np.zeros((self.frame_height, self.frame_width), dtype=np.uint8)
        self.edges_display = ax2.imshow(dummy_edges, cmap='gray', aspect='auto')

        # === 제어 대시보드 (좌하) ===
        ax3.set_facecolor('black')
        ax3.set_title('🤖 AI Control Dashboard',
                     fontsize=16, color='white', fontweight='bold')
        ax3.set_xlim(0, 10)
        ax3.set_ylim(0, 10)

        self.control_text = ax3.text(0.5, 5, '', fontsize=11, color='cyan',
                                    family='monospace', ha='left', va='center')
        ax3.axis('off')

        # === 성능 그래프 (우하) ===
        ax4.set_facecolor('black')
        ax4.set_title('📈 Performance Metrics',
                     fontsize=14, color='white', fontweight='bold')
        ax4.set_xlabel('Time Steps', color='white')
        ax4.set_ylabel('Values', color='white')
        ax4.grid(True, alpha=0.3)

        # 기준선
        ax4.axhline(y=0, color='cyan', linestyle='-', alpha=0.5, label='Perfect Tracking')

        self.error_plot, = ax4.plot([], [], 'red', linewidth=2, label='Lane Error')
        self.steering_plot, = ax4.plot([], [], 'yellow', linewidth=2, label='Steering Angle')
        ax4.legend()
        ax4.set_xlim(0, 100)
        ax4.set_ylim(-100, 100)

        return fig, ax1, ax2, ax3, ax4

    def animate(self, frame):
        """애니메이션 업데이트"""
        # 1. 현재 도로 영상 가져오기
        if self.road_images:
            current_frame = self.road_images[self.current_image_index].copy()
        else:
            current_frame = self.create_dummy_road_image()

        # 2. 차선 검출
        lane_info = self.detect_lanes_in_frame(current_frame)

        # 3. PID 제어
        error = self.pid_controller()

        # 4. 가상 차량 업데이트
        self.update_virtual_vehicle()

        # 5. 기록
        self.positions.append([self.car_x, self.car_y])
        self.steering_history.append(self.steering_angle)
        self.lane_errors.append(error)

        # 기록 길이 제한
        max_history = 100
        if len(self.positions) > max_history:
            self.positions.pop(0)
            self.steering_history.pop(0)
            self.lane_errors.pop(0)

        # 6. 시각화 업데이트

        # 실제 영상 + 오버레이
        overlay_frame = self.draw_lane_detection_overlay(current_frame)
        # BGR to RGB 변환 (matplotlib용)
        overlay_rgb = cv2.cvtColor(overlay_frame, cv2.COLOR_BGR2RGB)
        self.video_display.set_array(overlay_rgb)

        # 엣지 검출 결과
        if 'roi_edges' in lane_info:
            self.edges_display.set_array(lane_info['roi_edges'])

        # 제어 대시보드
        lane_status = "✅ Detected" if (lane_info.get('left_lane') or lane_info.get('right_lane')) else "❌ Not Found"

        control_info = f"""🤖 AI AUTONOMOUS DRIVING

📹 VIDEO ANALYSIS
Lane Detection: {lane_status}
Target X: {self.target_lane_x:.0f} px

🚗 VEHICLE STATUS
Position X: {self.car_x:.1f} px
Position Y: {self.car_y:.1f} px
Speed: {self.car_speed:.1f} m/s

🎛️ CONTROL SYSTEM
Steering: {self.steering_angle:.1f}°
Lane Error: {error:.1f} px
Control Mode: {"ACTIVE" if abs(error) > 5 else "STABLE"}

📊 PID COMPONENTS
P: {self.kp * error:.1f}
I: {self.ki * self.integral_error:.1f}
D: {self.kd * (error - self.previous_error)/0.1:.1f}

⏱️ Frame: {frame}
        """
        self.control_text.set_text(control_info)

        # 성능 그래프
        if len(self.lane_errors) > 1:
            time_steps = list(range(len(self.lane_errors)))
            self.error_plot.set_data(time_steps, self.lane_errors)
            # 조향각을 스케일 조정해서 같은 그래프에 표시
            scaled_steering = [s*2 for s in self.steering_history]  # 2배 확대
            self.steering_plot.set_data(time_steps, scaled_steering)

        return [self.video_display, self.edges_display, self.control_text,
                self.error_plot, self.steering_plot]

# === 실행 ===
print("📹 Real Road Video Simulation 시작!")
print("🚗 실제 도로 영상에서 AI 자율주행 테스트!")

# 시뮬레이터 생성 및 이미지 다운로드
simulator = RealVideoLaneSimulator()
simulator.download_sample_road_images()

# 시각화 생성
fig, ax1, ax2, ax3, ax4 = simulator.create_real_video_visualization()

# 애니메이션 생성
anim = animation.FuncAnimation(
    fig, simulator.animate,
    interval=200,      # 200ms (더 천천히, 처리 시간 확보)
    frames=1000,       # 충분한 프레임
    blit=False,
    repeat=True
)

plt.tight_layout()
plt.show()

# MP4 저장
def save_real_video_simulation():
    print("\n💾 실제 도로 영상 시뮬레이션 저장 중...")
    try:
        Writer = animation.writers['ffmpeg']
        writer = Writer(fps=5, bitrate=3000)  # 고화질
        anim.save('real_road_video_simulation.mp4', writer=writer)

        from google.colab import files
        files.download('real_road_video_simulation.mp4')
        print("✅ 실제 도로 영상 시뮬레이션 저장 완료!")
    except Exception as e:
        print(f"❌ 저장 실패: {e}")

save_real_video_simulation()

print("\n📹 실제 도로 영상 시뮬레이션 특징:")
print("✅ 진짜 도로 이미지에서 차선 검출")
print("✅ 실시간 엣지 검출 결과 표시")
print("✅ 가상 차량의 AI 제어")
print("✅ PID 제어로 정확한 차선 추종")
print("✅ 4분할 화면으로 모든 정보 표시")
print("✅ 실제 자율주행차와 동일한 원리")
print("\n🚀 테슬라 오토파일럿과 같은 방식입니다!")